<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/qa-generation/data_designer_sdk_qa_pairs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 📚 Data Designer SDK: Generate Diverse Q&A Pairs

This notebook demonstrates a general approach for extracting Q&A pairs from any source document
(e.g. text, markdown, or PDF files). The generated Q&A pairs can be used for:
- **Instruction Tuning:** Training models with clear, self-contained examples.
- **Retrieval-Augmented Generation (RAG):** Enhancing retrieval systems with precise and context-supported Q&A pairs.
- **Search and FAQ Systems:** Powering natural language query systems and documentation.

> **Note:** The [Data Designer](https://docs.gretel.ai/create-synthetic-data/gretel-data-designer-beta) functionality demonstrated in this notebook is currently in **Early Preview**. To access these features and run this notebook, please [join the waitlist](https://gretel.ai/navigator/data-designer#waitlist).

# 📘 Getting Started

First, let's install and import the required packages:

In [ ]:
# Install required libraries

%%capture
!pip install -qq langchain unstructured[pdf] smart_open git+https://github.com/gretelai/gretel-python-client

In [ ]:
# Configuration
# -------------
# Define your source document(s) and the number of Q&A pairs to generate.
# You can replace this with your own documents in PDF, markdown, or text formats.

DOCUMENT_LIST = ["https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/rag_evals/databricks-state-of-data-ai-report.pdf"]
NUM_QA_PAIRS = 50

In [ ]:
# Document Processing
# ------------------
# The DocumentProcessor class handles loading and chunking source documents for RAG evaluation.
# We use langchain's RecursiveCharacterTextSplitter and unstructured.io for robust document parsing.

from typing import List, Union
from langchain.text_splitter import RecursiveCharacterTextSplitter
from unstructured.partition.auto import partition
from smart_open import open
import tempfile
import os

class DocumentProcessor:
    def __init__(self, chunk_size: int = 4192, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def parse_document(self, uri: str) -> str:
        """Parse a single document from URI into raw text."""
        with open(uri, 'rb') as file:
            content = file.read()
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(content)
                temp_file.flush()
                elements = partition(temp_file.name)

        os.unlink(temp_file.name)
        return "\n\n".join([str(element) for element in elements])

    def process_documents(self, uris: Union[str, List[str]]) -> List[str]:
        """Process one or more documents into chunks for RAG evaluation."""
        if isinstance(uris, str):
            uris = [uris]

        all_chunks = []
        for uri in uris:
            text = self.parse_document(uri)
            chunks = self.text_splitter.split_text(text)
            all_chunks.extend(chunks)

        return all_chunks

In [ ]:
# Data Models
# -----------
# Define Pydantic models for structured output generation:
# 1. QAPair: Schema for question-answer pairs
# 2. EvalMetrics: Schema for scoring generation quality

from pydantic import BaseModel, Field
from typing import Optional, Literal

class QAPair(BaseModel):
    question: str = Field(..., description="A clear and concise question derived from the context.")
    answer: str = Field(..., description="A detailed and accurate answer fully supported by the context.")


class EvalMetrics(BaseModel):
    clarity: int = Field(
        ...,
        description="How clear and understandable is the question? (1=vague/confusing, 5=perfectly clear and well-structured)",
        ge=1,
        le=5
    )
    factual_accuracy: int = Field(
        ...,
        description="Is the answer fully supported by the context? (1=contains errors/unsupported claims, 5=completely accurate and supported)",
        ge=1,
        le=5
    )
    comprehensiveness: int = Field(
        ...,
        description="Does the answer cover all necessary details? (1=missing crucial information, 5=complete coverage of all relevant points)",
        ge=1,
        le=5
    )
    answer_relevance: int = Field(
        ...,
        description="Does the answer directly address what was asked in the question? (1=misaligned/tangential, 5=perfectly addresses the question)",
        ge=1,
        le=5
    )

In [ ]:
# Setup & Configure Data Designer
# --------------------------------
# Initialize the Data Designer with a custom system message to ensure that the generated
# Q&A pairs are high-quality, context-supported, and tailored for a variety of downstream applications.


from gretel_client.navigator import DataDesigner

# Process document chunks
processor = DocumentProcessor(chunk_size=4192, chunk_overlap=200)
chunks = processor.process_documents(DOCUMENT_LIST)

# Initialize Data Designer
designer = DataDesigner(
    api_key="prompt",
    model_suite="llama-3.x",  # or "apache-2.0" as needed
    endpoint="https://api.gretel.cloud",
    special_system_instructions="""\
You are an expert at generating high-quality, context-supported Q&A pairs.
Your output should be clear, concise, and factually correct.
Ensure that every question is self-contained and every answer is comprehensive and derived solely from the provided context.
"""
)

# Add Seed Columns for Controlled Diversity

# Context: Document chunks.
designer.add_categorical_seed_column(
    name="context",
    description="A chunk of text extracted from the source document.",
    values=chunks
)

# Difficulty with nested Sophistication.
designer.add_categorical_seed_column(
    name="difficulty",
    description="The overall difficulty level of the question.",
    values=["easy", "medium", "hard"],
    subcategories=[
        {
            "name": "sophistication",
            "values": {
                "easy": ["basic", "straightforward"],
                "medium": ["intermediate", "moderately complex"],
                "hard": ["advanced", "sophisticated"]
            }
        }
    ]
)

# Question Style: Tone and approach.
designer.add_categorical_seed_column(
    name="question_style",
    description="The style or tone of the question.",
    values=["factual", "exploratory", "analytical", "comparative"]
)

# Target Audience: Language complexity.
designer.add_categorical_seed_column(
    name="target_audience",
    description="The intended audience for the Q&A pair.",
    values=["novice", "intermediate", "expert"]
)

# Response Format: Expected answer style.
designer.add_categorical_seed_column(
    name="response_format",
    description="The format of the answer expected (e.g., short, detailed, step-by-step).",
    values=["short", "detailed", "step-by-step", "list"]
)

# Define Generation Template for Q&A Pairs
designer.add_generated_data_column(
    name="qa_pair",
    generation_prompt=(
        "\n{context}\n\n"
        "Based on the above context, generate a high-quality Q&A pair. The question should be clear and concise, "
        "and tailored for an audience at the '{target_audience}' level. The overall difficulty should be '{difficulty}', "
        "with a corresponding sophistication level (e.g., {sophistication}). The question style should be '{question_style}', "
        "and the answer should be provided in a '{response_format}' format. "
        "Ensure that the answer is factually accurate, fully derived from the context, and comprehensive.\n"
        "Put your thoughts within <think>...</think> before providing the JSON."
    ),
    data_config={
        "type": "structured",
        "params": {"model": QAPair}
    }
)

# Optional: Define Evaluation Template for Q&A Pairs
designer.add_generated_data_column(
    name="eval_metrics",
    llm_type="judge",
    generation_prompt=(
        "\n{context}\n\n"
        "For the above Q&A pair:\n"
        "{qa_pair}\n\n"
        "Rate each criterion on a scale of 1-5:\n"
        "- Clarity\n"
        "- Factual Accuracy\n"
        "- Comprehensiveness\n"
        "- Answer Relevance\n"
        "Put your thoughts within <think>...</think> before providing the JSON."
    ),
    data_config={
        "type": "structured",
        "params": {"model": EvalMetrics}
    }
)

# Preview a Sample of 10 Generated Records
preview = designer.generate_dataset_preview()
preview.display_sample_record()


In [ ]:
# Generate a report analyzing RAG coverage
# ---------------------

import json
from rich.console import Console
from rich.table import Table
from collections import Counter
import pandas as pd

def generate_qa_report(df):

    qa_df = pd.json_normalize(
        df.assign(eval_metrics=lambda _df: _df["eval_metrics"].apply(json.loads)).to_dict(orient="records")
    )
    console = Console()
    categories = ['difficulty', 'sophistication', 'question_style', 'target_audience', 'response_format']

    console.print("\n[bold blue]🧠 QA Pair Generation Report[/bold blue]", justify="center")
    console.print("=" * 80, justify="center")
    console.print(f"\n[bold]Total QA Pairs:[/bold] {len(qa_df)}")

    for category in categories:
        if category in qa_df.columns:
            # Only count non-empty values
            counts = Counter(x for x in qa_df[category] if pd.notna(x) and x != '')
            if not counts:
                continue

            table = Table(title=f"\n{category.title()} Distribution")
            table.add_column("Category", style="cyan")
            table.add_column("Count", justify="right")
            table.add_column("Percentage", justify="right")

            total = sum(counts.values())
            for value, count in sorted(counts.items()):
                percentage = (count / total) * 100
                table.add_row(str(value), str(count), f"{percentage:.1f}%")

            console.print(table)

    if any(col.startswith('eval_metrics.') for col in qa_df.columns):
        metrics_table = Table(title="\nQuality Metrics Summary")
        metrics_table.add_column("Metric")
        metrics_table.add_column("Average Score", justify="right")

        metrics = ['clarity', 'factual_accuracy', 'comprehensiveness', "answer_relevance"]
        for metric in metrics:
            col = f'eval_metrics.{metric}'
            if col in qa_df.columns:
                scores = qa_df[col].dropna()
                if len(scores) > 0:
                    avg_score = scores.mean()
                    metrics_table.add_row(
                        metric.replace('_', ' ').title(),
                        f"{avg_score:.2f}/5.00"
                    )

        console.print(metrics_table)

generate_qa_report(preview.dataset)

In [ ]:
# Explore the generated preview as a Pandas DataFrame
# ---------------------------

preview.dataset

In [ ]:
# Generate and Analyze Dataset
# ---------------------------
# Uncomment these lines to generate evaluation pairs, save them to JSONL,
# and analyze the coverage and quality of the generated dataset.

#batch_job = designer.submit_batch_workflow(num_records=NUM_QA_PAIRS)
#dataset = batch_job.fetch_dataset(wait_for_completion=True)

#generate_qa_report(dataset)

#dataset.to_json('qa_pairs.jsonl', orient='records', lines=True)
